In [3]:
#import necessary libraries
import pandas as pd
import csv
import re
import validators
import emoji
import unidecode
import nltk
import pickle
nltk.download('stopwords')
import numpy as np
from sklearn.model_selection import train_test_split
from nltk.tokenize import WhitespaceTokenizer
from itertools import chain
import time
from sklearn.feature_extraction.text import TfidfVectorizer
import keras
from keras.preprocessing import sequence
import tensorflow as tf
from keras import Input
from keras import optimizers
from keras import backend as K
import torch
from keras import regularizers
from keras.utils import np_utils
from sklearn.preprocessing import LabelEncoder
from sklearn.datasets import make_blobs
from keras.models import Sequential
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from keras.layers import Dense, Activation, Dropout, Flatten
from keras.layers import Embedding, Conv1D, MaxPooling1D, GlobalMaxPooling1D 
from keras.utils import plot_model
from keras.preprocessing import sequence
from keras.preprocessing.text import Tokenizer
from keras.callbacks import EarlyStopping
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt
from matplotlib import pyplot
from keras.models import load_model
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.feature_selection import SelectPercentile, f_classif

import tensorflow as tf
from tensorflow import keras
from keras import optimizers
from keras import regularizers
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout, Flatten
from keras.layers import Embedding, Conv1D, MaxPooling1D, GlobalMaxPooling1D 
from keras.utils import np_utils
from keras.utils import plot_model
from keras.preprocessing import sequence
from keras.preprocessing.text import Tokenizer
from itertools import chain

from tqdm import tqdm
from gensim.models import fasttext
from gensim.test.utils import datapath
import os, re, csv, math, codecs, pickle, nltk
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

[nltk_data] Downloading package stopwords to C:\Users\Kirby
[nltk_data]     Wenceslao\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [4]:
#Load test and testresutls   

with open('x_test.pkl', 'rb') as file:
    x_test = pickle.load(file)
    
with open('y_test.pkl', 'rb') as file:
    y_test = pickle.load(file)
    
with open('fastTextCNN_test_results.pkl', 'rb') as file:
    fastTextCNN_test_results = pickle.load(file) 
    
with open('TFIDFFFNN_test_results.pkl', 'rb') as file:
    TFIDFFFNN_test_results = pickle.load(file)  
    
#Convert Labels from Strings to categorical Integers {Non-Hate = 1, Hate = 0}


df_ytest = pd.DataFrame(y_test, columns = ['Label'])

fastTextCNN_test_results.columns = ['Tokens', 'Label', 'Prediction']
fastTextCNN_test_results['Label'] = fastTextCNN_test_results['Label'].str.get(0)
fastTextCNN_test_results['Prediction'] = fastTextCNN_test_results['Prediction'].str.get(0)

fastTextCNN_tweets_joined = []

for i in range(len(fastTextCNN_test_results.Tokens.values)):
    tweet = " ".join(fastTextCNN_test_results.Tokens.values[i])
    fastTextCNN_tweets_joined.append(tweet)
    
fastTextCNN_test_results.insert(0, "Tweet", fastTextCNN_tweets_joined, True)


TFIDFFFNN_test_results.columns = ['Tokens', 'Label', 'Prediction']

tfidfffnn_tweets_joined = []
for i in range(len(TFIDFFFNN_test_results.Tokens.values)):
    tweet = " ".join(TFIDFFFNN_test_results.Tokens.values[i])
    tfidfffnn_tweets_joined.append(tweet)
    
TFIDFFFNN_test_results.insert(0, "Tweet", tfidfffnn_tweets_joined, True)
#TFIDFFFNN_test_results['Label'] = TFIDFFFNN_test_results['Label'].str.get(0)
#TFIDFFFNN_test_results['Prediction'] = TFIDFFFNN_test_results['Prediction'].str.get(0)
#flatten training set for fitting on tfidf
#x_train_fit = list(chain.from_iterable(x_train))

mapping = {'Non-hate': 0, 'Hate': 1}
mapping_reverse = {0: 'Non-hate', 1: 'Hate'}
fastTextCNN_test_results = fastTextCNN_test_results.replace({'Label': mapping_reverse})
TFIDFFFNN_test_results = TFIDFFFNN_test_results.replace({'Label': mapping_reverse})
df_ytest = df_ytest.replace({'Label': mapping})

In [5]:
def dummy_fun(doc):
    return doc

def get_keywords(data,filename):
    print(filename)
    tfidf = TfidfVectorizer(
        analyzer='word',
        tokenizer=dummy_fun,
        preprocessor=dummy_fun,
        token_pattern=None)  

    #fit text to tfidf
    tfidf.fit_transform(data)


    fasttext_dict = list(zip(tfidf.get_feature_names(), tfidf.idf_))
    fasttext_df = pd.DataFrame(fasttext_dict, columns = ['Token', 'TFIDF_Value'])
    fasttextresults_label_sorted = fasttext_df.sort_values(by=['TFIDF_Value'], ascending=False)
    fasttextresults_label_sorted.to_csv(filename, index=False)
    
    return fasttextresults_label_sorted

In [6]:
TFIDFFFNN_test_results.to_csv('TFIDFFFNN_test_results.csv', index=False)
TFIDFFFNN_test_results

,Tweet,Tokens,Label,Prediction
0,gang dear let remind ocho derecho also make re...,"[gang, dear, let, remind, ocho, derecho, also,...",Hate,0.999999
1,gobyernong tapat angat buhay presidente bise-p...,"[gobyernong, tapat, angat, buhay, presidente, ...",Non-hate,0.999998
2,okay ara sakyan du vibes namon asked president...,"[okay, ara, sakyan, du, vibes, namon, asked, p...",Hate,0.931061
3,president cutie,"[president, cutie]",Non-hate,0.999998
4,president malakas daw nag mudmod pera laki bab...,"[president, malakas, daw, nag, mudmod, pera, l...",Hate,0.999999
...,...,...,...,...
1019,president vice president kiko pangilinan,"[president, vice, president, kiko, pangilinan]",Non-hate,0.990039
1020,sir today,"[sir, today]",Non-hate,0.794478
1021,marcosjr duwag ayaw umattend debate kayang ipa...,"[marcosjr, duwag, ayaw, umattend, debate, kaya...",Hate,1.000000
1022,lumalabas pagka komunista sinasagot bbm si pal...,"[lumalabas, pagka, komunista, sinasagot, bbm, ...",Hate,0.978872


In [7]:
groupedby_ffnn_label = TFIDFFFNN_test_results.groupby(TFIDFFFNN_test_results.Label)

#group by label
ffnn_nonhate = groupedby_ffnn_label.get_group("Non-hate")
ffnn_hate = groupedby_ffnn_label.get_group("Hate")

In [8]:
#tfidf highest keywords ffnn hate
get_keywords(ffnn_hate.Tokens.values,'TFIDFFFNN_hate_keywords.csv').head(20)

TFIDFFFNN_hate_keywords.csv


,Token,TFIDF_Value
0,aaral,6.56452
1932,naramdaman,6.56452
1911,nananahimik,6.56452
1913,nandaya,6.56452
1914,nandito,6.56452
1915,nandoon,6.56452
1917,nanggaling,6.56452
1918,nanghinayang,6.56452
1919,nanguna,6.56452
1921,nangyayari,6.56452


In [9]:
#tfidf highest keywords ffnn nonhate
get_keywords(ffnn_nonhate.Tokens.values,'TFIDFFFNN_nonhate_keywords.csv').head(20)

TFIDFFFNN_nonhate_keywords.csv


,Token,TFIDF_Value
0,aasa,6.529429
1911,sour,6.529429
1921,spending,6.529429
1922,spent,6.529429
1142,malumanay,6.529429
1141,maloko,6.529429
1140,maling,6.529429
1139,maliit,6.529429
1138,malayong,6.529429
1923,spikers,6.529429


In [10]:
#hate
ffnn_hate

,Tweet,Tokens,Label,Prediction
0,gang dear let remind ocho derecho also make re...,"[gang, dear, let, remind, ocho, derecho, also,...",Hate,0.999999
2,okay ara sakyan du vibes namon asked president...,"[okay, ara, sakyan, du, vibes, namon, asked, p...",Hate,0.931061
4,president malakas daw nag mudmod pera laki bab...,"[president, malakas, daw, nag, mudmod, pera, l...",Hate,0.999999
5,like vp ideas silang magagandang layunin ever ...,"[like, vp, ideas, silang, magagandang, layunin...",Hate,0.706385
6,naman talaga aayusin iyan tignan naman kakampi...,"[naman, talaga, aayusin, iyan, tignan, naman, ...",Hate,0.972121
...,...,...,...,...
1017,philippines presidential frontrunner using tik...,"[philippines, presidential, frontrunner, using...",Hate,1.000000
1018,pangilinan kiko win war lies disinformation mi...,"[pangilinan, kiko, win, war, lies, disinformat...",Hate,0.996715
1021,marcosjr duwag ayaw umattend debate kayang ipa...,"[marcosjr, duwag, ayaw, umattend, debate, kaya...",Hate,1.000000
1022,lumalabas pagka komunista sinasagot bbm si pal...,"[lumalabas, pagka, komunista, sinasagot, bbm, ...",Hate,0.978872


In [11]:
#top ffnn hate sentences sorted by prediction
ffnn_hate_sortedbyprediction = ffnn_hate.sort_values(by=['Prediction'], ascending=False)
ffnn_hate_sortedbyprediction.to_csv('TFIDFFFNN_hate_results.csv', index=False)
ffnn_hate_sortedbyprediction

,Tweet,Tokens,Label,Prediction
614,marcosjr duwag ayaw umattend debate kayang ipa...,"[marcosjr, duwag, ayaw, umattend, debate, kaya...",Hate,1.000000
788,nagsasabing bobo si gago mas bobo kayo hahahah...,"[nagsasabing, bobo, si, gago, mas, bobo, kayo,...",Hate,1.000000
125,ibiboto sana kaso nalaman pala worst vp bansan...,"[ibiboto, sana, kaso, nalaman, pala, worst, vp...",Hate,1.000000
170,philippines presidential frontrunner using tik...,"[philippines, presidential, frontrunner, using...",Hate,1.000000
210,ekonomista gago diba bumagsak daw econ,"[ekonomista, gago, diba, bumagsak, daw, econ]",Hate,1.000000
...,...,...,...,...
672,ah see context itong rappler fact check post f...,"[ah, see, context, itong, rappler, fact, check...",Hate,0.537786
181,tuwing eleksiyon kami'y binibigyan lang laya e...,"[tuwing, eleksiyon, kami'y, binibigyan, lang, ...",Hate,0.532094
306,light penetrates darkness also want new world ...,"[light, penetrates, darkness, also, want, new,...",Hate,0.527762
738,tapatang inaabangan presidential debate media ...,"[tapatang, inaabangan, presidential, debate, m...",Hate,0.510994


In [12]:
#for nonhate
ffnn_nonhate

,Tweet,Tokens,Label,Prediction
1,gobyernong tapat angat buhay presidente bise-p...,"[gobyernong, tapat, angat, buhay, presidente, ...",Non-hate,0.999998
3,president cutie,"[president, cutie]",Non-hate,0.999998
13,walaaaa mayyy panaloo putanginaaaaaa mahal mahal,"[walaaaa, mayyy, panaloo, putanginaaaaaa, maha...",Non-hate,0.999862
15,manifestations workk time manifest president,"[manifestations, workk, time, manifest, presid...",Non-hate,0.999057
16,hated despise naniwala election rigged told na...,"[hated, despise, naniwala, election, rigged, t...",Non-hate,0.844684
...,...,...,...,...
1013,vice president stepped thursday announce decis...,"[vice, president, stepped, thursday, announce,...",Non-hate,0.958783
1014,former national security adviser warns pink me...,"[former, national, security, adviser, warns, p...",Non-hate,0.949715
1015,good morning philippines voting wishing kakamp...,"[good, morning, philippines, voting, wishing, ...",Non-hate,0.996845
1019,president vice president kiko pangilinan,"[president, vice, president, kiko, pangilinan]",Non-hate,0.990039


In [13]:
#top ffnn nonhate sentences sorted by prediction
ffnn_nonhate_sortedbyprediction = ffnn_nonhate.sort_values(by=['Prediction'], ascending=False)
ffnn_nonhate_sortedbyprediction.to_csv('TFIDFFFNN_nonhate_results.csv', index=False)
ffnn_nonhate_sortedbyprediction

,Tweet,Tokens,Label,Prediction
661,ngayong darating halalan tatanglaw buong bayan...,"[ngayong, darating, halalan, tatanglaw, buong,...",Non-hate,1.000000
653,ngayong darating halalan tatanglaw buong bayan...,"[ngayong, darating, halalan, tatanglaw, buong,...",Non-hate,1.000000
188,ngayong darating halalan tatanglaw buong bayan...,"[ngayong, darating, halalan, tatanglaw, buong,...",Non-hate,1.000000
580,tara ipakita buong pagsuporta mabuting pinuno ...,"[tara, ipakita, buong, pagsuporta, mabuting, p...",Non-hate,1.000000
568,tara ipakita buong pagsuporta mabuting pinuno ...,"[tara, ipakita, buong, pagsuporta, mabuting, p...",Non-hate,1.000000
...,...,...,...,...
838,wag po nating hanapin yung ayaw tayong harapin...,"[wag, po, nating, hanapin, yung, ayaw, tayong,...",Non-hate,0.521391
891,let's watch tunay public servant like vp nagawa,"[let's, watch, tunay, public, servant, like, v...",Non-hate,0.519012
225,tops latest octa research pulso pilipino surve...,"[tops, latest, octa, research, pulso, pilipino...",Non-hate,0.518527
821,wealth completely refute marcoses denials ill-...,"[wealth, completely, refute, marcoses, denials...",Non-hate,0.507541


In [14]:
#For fasttext results

In [15]:
fastTextCNN_test_results.to_csv('fastTextCNN_test_results.csv', index=False)
fastTextCNN_test_results

,Tweet,Tokens,Label,Prediction
0,gang dear let remind ocho derecho also make re...,"[gang, dear, let, remind, ocho, derecho, also,...",Hate,0.933314
1,gobyernong tapat angat buhay presidente bise-p...,"[gobyernong, tapat, angat, buhay, presidente, ...",Non-hate,0.001019
2,okay ara sakyan du vibes namon asked president...,"[okay, ara, sakyan, du, vibes, namon, asked, p...",Non-hate,0.048966
3,president cutie,"[president, cutie]",Non-hate,0.009071
4,president malakas daw nag mudmod pera laki bab...,"[president, malakas, daw, nag, mudmod, pera, l...",Hate,0.970548
...,...,...,...,...
1019,president vice president kiko pangilinan,"[president, vice, president, kiko, pangilinan]",Non-hate,0.000999
1020,sir today,"[sir, today]",Non-hate,0.126034
1021,marcosjr duwag ayaw umattend debate kayang ipa...,"[marcosjr, duwag, ayaw, umattend, debate, kaya...",Hate,0.975011
1022,lumalabas pagka komunista sinasagot bbm si pal...,"[lumalabas, pagka, komunista, sinasagot, bbm, ...",Hate,0.715232


In [16]:
groupedby_cnn_label = fastTextCNN_test_results.groupby(fastTextCNN_test_results.Label)
#group by label
cnn_nonhate = groupedby_cnn_label.get_group("Non-hate")
cnn_hate = groupedby_cnn_label.get_group("Hate")

In [17]:
#get top keywords by tfidf

In [18]:
#fasttextcnn highest keywords nonhate
get_keywords(cnn_nonhate.Tokens.values,'fastTextCNN_nonhate_keywords.csv').head(20)

fastTextCNN_nonhate_keywords.csv


,Token,TFIDF_Value
0,aasa,6.879135
2026,natatamaan,6.879135
2017,nararamdaman,6.879135
2018,nararapat,6.879135
2019,narinig,6.879135
2021,nasabi,6.879135
2022,natagalan,6.879135
2023,natalo,6.879135
2024,natapos,6.879135
2025,natatalo,6.879135


In [19]:
#fasttextcnn highest keywords hate
get_keywords(cnn_hate.Tokens.values,'fastTextCNN_hate_keywords.csv').head(20)

fastTextCNN_hate_keywords.csv


,Token,TFIDF_Value
0,aaral,6.046646
1285,naninira,6.046646
1314,naupo,6.046646
1313,native,6.046646
1312,natitira,6.046646
1311,nation,6.046646
1310,nating,6.046646
1309,natin,6.046646
1308,natatawa,6.046646
1307,natapos,6.046646


In [20]:
#top cnn hate sentences sorted by prediction
cnn_hate_sortedbyprediction = cnn_hate.sort_values(by=['Prediction'], ascending=False)
cnn_hate_sortedbyprediction.to_csv('fastTextCNN_hate_results.csv', index=False)
cnn_hate_sortedbyprediction

,Tweet,Tokens,Label,Prediction
788,nagsasabing bobo si gago mas bobo kayo hahahah...,"[nagsasabing, bobo, si, gago, mas, bobo, kayo,...",Hate,0.999999
969,ba nareminds katangahan kabobohan gustong bobo...,"[ba, nareminds, katangahan, kabobohan, gustong...",Hate,0.999981
808,capability credibility ampota seryoso ganyan k...,"[capability, credibility, ampota, seryoso, gan...",Hate,0.999960
328,ulol yung nag sasabing i'm inc vote ulol nilok...,"[ulol, yung, nag, sasabing, i'm, inc, vote, ul...",Hate,0.999385
177,pota ba nadeklara nyan ulol,"[pota, ba, nadeklara, nyan, ulol]",Hate,0.999245
...,...,...,...,...
143,two-horse race philippine presidency emerged a...,"[two-horse, race, philippine, presidency, emer...",Hate,0.513174
36,twitter party official hashtag,"[twitter, party, official, hashtag]",Hate,0.510301
736,umattend lang sana si edi sana trending katang...,"[umattend, lang, sana, si, edi, sana, trending...",Hate,0.508638
426,matutuwa ba sinabihan nakikita sayo si grabee ...,"[matutuwa, ba, sinabihan, nakikita, sayo, si, ...",Hate,0.506771


In [21]:
cnn_nonhate

,Tweet,Tokens,Label,Prediction
1,gobyernong tapat angat buhay presidente bise-p...,"[gobyernong, tapat, angat, buhay, presidente, ...",Non-hate,0.001019
2,okay ara sakyan du vibes namon asked president...,"[okay, ara, sakyan, du, vibes, namon, asked, p...",Non-hate,0.048966
3,president cutie,"[president, cutie]",Non-hate,0.009071
5,like vp ideas silang magagandang layunin ever ...,"[like, vp, ideas, silang, magagandang, layunin...",Non-hate,0.023150
6,naman talaga aayusin iyan tignan naman kakampi...,"[naman, talaga, aayusin, iyan, tignan, naman, ...",Non-hate,0.367104
...,...,...,...,...
1015,good morning philippines voting wishing kakamp...,"[good, morning, philippines, voting, wishing, ...",Non-hate,0.028516
1016,presidential debates cnn philippines president...,"[presidential, debates, cnn, philippines, pres...",Non-hate,0.054334
1018,pangilinan kiko win war lies disinformation mi...,"[pangilinan, kiko, win, war, lies, disinformat...",Non-hate,0.242651
1019,president vice president kiko pangilinan,"[president, vice, president, kiko, pangilinan]",Non-hate,0.000999


In [22]:
#top cnn nonhate sentences sorted by prediction
cnn_nonhate_sortedbyprediction = cnn_nonhate.sort_values(by=['Prediction'], ascending=True)
cnn_nonhate_sortedbyprediction.to_csv('fastTextCNN_nonhate_results.csv', index=False)
cnn_nonhate_sortedbyprediction

,Tweet,Tokens,Label,Prediction
428,president bise-president kiko pangilinan beech...,"[president, bise-president, kiko, pangilinan, ...",Non-hate,0.000428
151,mandirigmang tumitindig nakikiisa pagsuporta k...,"[mandirigmang, tumitindig, nakikiisa, pagsupor...",Non-hate,0.000519
603,gobyernong tapat angat buhay si volleydeity bi...,"[gobyernong, tapat, angat, buhay, si, volleyde...",Non-hate,0.000623
581,gobyernong tapat angat buhay si shaira kabataa...,"[gobyernong, tapat, angat, buhay, si, shaira, ...",Non-hate,0.000654
275,gobyernong tapat angat buhay si vince michael ...,"[gobyernong, tapat, angat, buhay, si, vince, m...",Non-hate,0.000707
...,...,...,...,...
848,lamia troll aning bbm supporters ug boycott oi...,"[lamia, troll, aning, bbm, supporters, ug, boy...",Non-hate,0.490945
894,kawawa talaga popular celebrities use ops andr...,"[kawawa, talaga, popular, celebrities, use, op...",Non-hate,0.491632
747,shaddai endorses marcos-sara duterte tandem sh...,"[shaddai, endorses, marcos-sara, duterte, tand...",Non-hate,0.492644
222,long years pilipino nalang trapped di makamove...,"[long, years, pilipino, nalang, trapped, di, m...",Non-hate,0.493702
